<a href="https://colab.research.google.com/github/AhmedCoolProjects/PythonScraping/blob/main/ScrapingAvitoCarsData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests as rq
from bs4 import BeautifulSoup
import csv

In [162]:
def starting(page_link):
  headers = {"User-Agent":
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36 Edg/91.0.864.37"}
  page = rq.get(page_link,headers=headers)
  soup = BeautifulSoup(page.content, "html.parser")
  AVITO_STORE_DOMAIN_PREFIX = "https://www.avito.ma"
  salers_i = list(soup.find_all("a",{"id":"shop_link"}))
  selers_small = list(soup.find_all("div",{"class":"ctext3"}))
  # getting avito store links
  avito_stor_links = [AVITO_STORE_DOMAIN_PREFIX + i['href'] for i in salers_i]
  # getting store names
  stores_names = [i.get_text() for i in salers_i]
  # getting store city
  stores_cities = [i.select("small")[1].get_text() for i in selers_small]
  # getting nbr of articals for each seler
  selers_articals_nbr = [int(i.select('i')[1].get_text().split('\xa0')[0]) for i in selers_small]
  return avito_stor_links,stores_names,stores_cities,selers_articals_nbr

In [163]:
avito_stor_links,stores_names,stores_cities,selers_articals_nbr = starting("https://www.avito.ma/fr/boutiques/maroc/voitures-%C3%A0_vendre")

In [164]:
# function to map all the store links and get their infos
def getStorInfoByLink(index_):
  store_page = rq.get(avito_stor_links[index_],headers=headers)
  store_soup = BeautifulSoup(store_page.content, "html.parser")
  if store_soup.find("div",{"class":"mtl mbs"}):
    store_address = store_soup.find("div",{"class":"mtl mbs"}).get_text().replace('\n',"")
  else:
    store_address = None
  store_bien = store_soup.find('h2',{'class':" d-inline-block text-truncate"}).select('a')[0].get_text()
  if ( store_soup.find('span',{"class":'price_value'})):
    store_price = store_soup.find('span',{"class":'price_value'}).get_text() + " DH" 
  else:
    store_price = None
  saler_info = {"le_vendeur":stores_names[index_],
    'le_montant_de_la_vente':store_price,
    "le_bien_vendu":store_bien,
    "l_adresse_du_vendeur":store_address,
    "la_ville":stores_cities[index_],
    "vendeur_ecommorce_link":avito_stor_links[index_],
    "vendeur_nbr_d_article":selers_articals_nbr[index_]
    }
  return saler_info


In [ ]:
final_array = []
for i in range(len(avito_stor_links)):
  if i != 35 or i != 45:
    final_array.append(getStorInfoByLink(i))
  print(i)

In [168]:
avito_stor_links,stores_names,stores_cities,selers_articals_nbr = starting("https://www.avito.ma/fr/boutiques/maroc/voitures-%C3%A0_vendre?o=2")

In [ ]:
for i in range(len(avito_stor_links)):
  final_array.append(getStorInfoByLink(i))
  print(i)

In [ ]:
avito_stor_links,stores_names,stores_cities,selers_articals_nbr = starting("https://www.avito.ma/fr/boutiques/maroc/voitures-%C3%A0_vendre?o=3")

In [ ]:
for i in range(len(avito_stor_links)):
  final_array.append(getStorInfoByLink(i))
  print(i)

In [ ]:
keys = final_array[0].keys()
with open('voitures.csv', 'w', newline='')  as my_csv_file:
    dict_writer = csv.DictWriter(my_csv_file, fieldnames=keys)
    dict_writer.writeheader()
    dict_writer.writerows(final_array)
print("done")

In [173]:
import pandas as pd
df = pd.read_csv('/content/voitures.csv')
df.columns = [i.replace('_',' ') for i in df.columns]
df.dropna(inplace=True)
df.to_csv('final_voitures.csv')